In [12]:
import yfinance as yf

token = 'BTC-USD'
# start_date = '2015-01-01'

data = yf.download(token, interval='1d')
data.to_csv(f'{token}.csv')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,457.334015,21056800
2014-09-18,456.859985,456.859985,413.104004,424.440002,424.440002,34483200
2014-09-19,424.102997,427.834991,384.532013,394.795990,394.795990,37919700
2014-09-20,394.673004,423.295990,389.882996,408.903992,408.903992,36863600
2014-09-21,408.084991,412.425995,393.181000,398.821014,398.821014,26580100
...,...,...,...,...,...,...
2024-12-22,97218.320312,97360.265625,94202.187500,95104.937500,95104.937500,43147981314
2024-12-23,95099.390625,96416.210938,92403.132812,94686.242188,94686.242188,65239002919
2024-12-24,94684.343750,99404.062500,93448.015625,98676.093750,98676.093750,47114953674


In [13]:
import pandas as pd 

data = pd.read_csv(f'{token}.csv', parse_dates=['Date'], index_col=['Date'])
data.drop(['Adj Close', 'Open', 'High', 'Low', 'Volume'], axis=1, inplace=True)
data

,Close
Date,
2014-09-17,457.334015
2014-09-18,424.440002
2014-09-19,394.795990
2014-09-20,408.903992
2014-09-21,398.821014
...,...
2024-12-22,95104.937500
2024-12-23,94686.242188
2024-12-24,98676.093750


In [14]:
import pandas as pd
import talib

data['ema20'] = talib.EMA(data['Close'], timeperiod=20)
data['ema50'] = talib.EMA(data['Close'], timeperiod=50)
data['rsi'] = talib.RSI(data['Close'], timeperiod=14)
data['macd'], data['signal'], data['hist'] = talib.MACD(data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)

data['Price-Up'] = (data['Close'].shift(-1) > data['Close']).astype(int)
data.dropna(inplace=True)
data.to_csv('BTC-I.csv')
data

,Close,ema20,ema50,rsi,macd,signal,hist,Price-Up
Date,,,,,,,,
2014-11-05,339.485992,350.004907,372.766840,40.963965,-14.092868,-13.286073,-0.806796,1
2014-11-06,349.290009,349.936822,371.846180,46.003906,-12.408570,-13.110572,0.702002,0
2014-11-07,342.415009,349.220458,370.692017,43.217639,-11.495987,-12.787655,1.291668,1
2014-11-08,345.488007,348.864987,369.703624,44.826192,-10.404854,-12.311095,1.906241,1
2014-11-09,363.264008,350.236322,369.451090,53.102358,-8.013374,-11.451551,3.438177,1
...,...,...,...,...,...,...,...,...
2024-12-22,95104.937500,98692.014312,92400.356511,44.934433,1437.643624,2697.516320,-1259.872696,0
2024-12-23,94686.242188,98310.512205,92489.999087,44.163328,974.011455,2352.815347,-1378.803892,1
2024-12-24,98676.093750,98345.329495,92732.591034,52.524176,917.945789,2065.841435,-1147.895647,1


In [15]:
import pandas as pd 
import ta

# data['sma'] = ta.trend.sma_indicator(data['Close'], window=30)
data['ema20'] = ta.trend.ema_indicator(data['Close'], window=20)
data['ema50'] = ta.trend.ema_indicator(data['Close'], window=50)
# data['ema200'] = ta.trend.ema_indicator(data['Close'], window=200)


# Añadir RSI
data['rsi'] = ta.momentum.rsi(data['Close'], window=14)

# Añadir MACD
macd = ta.trend.MACD(data['Close'], window_slow=26, window_fast=12, window_sign=9)
data['macd'] = macd.macd()
data['macd_signal'] = macd.macd_signal()

#Precio Futuro
# data['Close_future'] = data['Close'].shift(-1)
# Dirección del Movimiento de Precio
data['Price_Up'] = (data['Close'].shift(-1) > data['Close']).astype(int)


#Boramos datos nulos
data.dropna(inplace=True)

data.to_csv('BTC-I.csv',index=['Date'])
data = pd.read_csv('BTC-I.csv', parse_dates=['Date'], index_col=['Date'])
data

,Close,ema20,ema50,rsi,macd,signal,hist,Price-Up,macd_signal,Price_Up
Date,,,,,,,,,,
2014-12-24,322.533997,339.324512,349.788445,39.191642,-11.030234,-10.692487,-0.702672,0,-10.160622,0
2014-12-25,319.007996,337.389606,348.581369,37.833239,-11.265775,-10.874731,-0.728977,1,-10.381653,1
2014-12-26,327.924011,336.488121,347.771276,43.194857,-10.610682,-10.884508,-0.039108,0,-10.427459,0
2014-12-27,315.863007,334.523824,346.519972,38.373529,-10.938645,-10.953292,-0.275135,1,-10.529696,1
2014-12-28,317.239014,332.877652,345.371699,39.207226,-10.961172,-11.008536,-0.220977,0,-10.615991,0
...,...,...,...,...,...,...,...,...,...,...
2024-12-22,95104.937500,98692.014312,92400.356511,44.934433,1437.643624,2697.516320,-1259.872696,0,2697.516320,0
2024-12-23,94686.242188,98310.512205,92489.999087,44.163328,974.011455,2352.815347,-1378.803892,1,2352.815347,1
2024-12-24,98676.093750,98345.329495,92732.591034,52.524176,917.945789,2065.841435,-1147.895647,1,2065.841435,1


In [16]:
features = ['Close', 'sma', 'ema20', 'ema50', 'ema200', 'rsi', 'macd', 'macd_signal']
target = ['Price_Up']
# Separar X e y
X = data[features]
y = data[target]


KeyError: "['sma', 'ema200'] not in index"

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01, shuffle= False)


ValueError: With n_samples=0, test_size=0.01 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


# Stream_Crypto_Data 

In [5]:
import json 
import websocket
import pandas as pd
import numpy as np

coin = 'btcusdt@kline_1m'
socket = 'wss://stream.binance.com:9443/stream?streams=' + coin
print(socket)


wss://stream.binance.com:9443/stream?streams=btcusdt@kline_1m


In [6]:
def extract_ohlcv(source):
    event_time = pd.to_datetime(source['data']['E'], unit='ms')
    open = source['data']['k']['o']
    high = source['data']['k']['h']
    low = source['data']['k']['l']
    close = source['data']['k']['c']   
    volume = source['data']['k']['v']

    data = {'Open': open, 'High': high, 'Low': low, 'Close': close, 'Volume': volume}

    df = pd.DataFrame(data, index=[event_time])
    df.index.name = 'timestamp'
    df = df.astype(float)
    df = df.reset_index()
    print(df)
    return df

def on_message(ws, message):
    message = json.loads(message)
    extract_ohlcv(message)

In [7]:
ws = websocket.WebSocketApp(socket, on_message=on_message)
ws.run_forever()

                timestamp     Open     High       Low     Close   Volume
0 2024-12-26 18:25:56.034  95800.0  95873.5  95777.43  95873.49  3.58631
                timestamp     Open     High       Low    Close   Volume
0 2024-12-26 18:25:58.022  95800.0  95873.5  95777.43  95873.5  3.58869
                timestamp     Open     High       Low     Close   Volume
0 2024-12-26 18:26:00.018  95800.0  95873.5  95777.43  95873.49  3.61952
                timestamp      Open      High       Low     Close  Volume
0 2024-12-26 18:26:00.018  95873.49  95873.49  95873.49  95873.49     0.0
                timestamp     Open     High       Low    Close   Volume
0 2024-12-26 18:26:02.025  95873.5  95873.5  95873.49  95873.5  0.00905
                timestamp     Open     High       Low     Close   Volume
0 2024-12-26 18:26:04.017  95873.5  95873.5  95873.49  95873.49  0.07443
                timestamp     Open     High       Low    Close   Volume
0 2024-12-26 18:26:06.017  95873.5  95884.4  95873.49 

True